In [1]:
# !pip3 install git+https://github.com/rongardF/tvdatafeed.git@e6f6aaa7de439ac6e454d9b26d2760ded8dc4923

from tvDatafeed import TvDatafeed, Interval

In [24]:
from helpers_db import get_engine, run_sql
engine = get_engine()

In [25]:
# queries
query_last_day = """SELECT DISTINCT a.symbol, a.exchange, count(t.dt)
                    FROM assets a LEFT JOIN daily_ticks t ON a.symbol = t.symbol
                    WHERE category = 'stock'
                    GROUP BY a.symbol, a.exchange HAVING count(t.dt) = 0"""
                    #WHERE t.dt < (DATE_TRUNC('day', now()) - '1 day'::interval)"""
                    #  WHERE a.category = 'stock'
assets = None
with engine.begin() as conn:
  #  SELECT symbol, exchange FROM assets WHERE category='stock'
  assets = run_sql(conn, query_last_day).mappings().all()
# securities['symbol']
assets

[{'symbol': 'AAOI', 'exchange': 'NASDAQ', 'count': 0},
 {'symbol': 'AAPL', 'exchange': 'NASDAQ', 'count': 0},
 {'symbol': 'ADP', 'exchange': 'NASDAQ', 'count': 0},
 {'symbol': 'ALB', 'exchange': 'NYSE', 'count': 0},
 {'symbol': 'AMZN', 'exchange': 'NASDAQ', 'count': 0},
 {'symbol': 'BABA', 'exchange': 'NYSE', 'count': 0},
 {'symbol': 'CI', 'exchange': 'NYSE', 'count': 0},
 {'symbol': 'CLS', 'exchange': 'NYSE', 'count': 0},
 {'symbol': 'CRDF', 'exchange': 'NASDAQ', 'count': 0},
 {'symbol': 'CSCO', 'exchange': 'NASDAQ', 'count': 0},
 {'symbol': 'CVNA', 'exchange': 'NYSE', 'count': 0},
 {'symbol': 'DBK', 'exchange': 'XETR', 'count': 0},
 {'symbol': 'ESEA', 'exchange': 'NASDAQ', 'count': 0},
 {'symbol': 'EUR.USD', 'exchange': 'NASDAQ', 'count': 0},
 {'symbol': 'GCT', 'exchange': 'NASDAQ', 'count': 0},
 {'symbol': 'GOOG', 'exchange': 'NASDAQ', 'count': 0},
 {'symbol': 'HEI', 'exchange': 'NYSE', 'count': 0},
 {'symbol': 'INMD', 'exchange': 'NASDAQ', 'count': 0},
 {'symbol': 'JPM', 'exchange'

In [23]:
from helpers_db import insert_on_conflict_nothing_ticks, truncate_table

tv = TvDatafeed()

def fetch_ticks(symbol, exchange, bars=100):
  df = tv.get_hist(
      symbol=symbol,
      exchange=exchange,
      interval=Interval.in_daily,
      n_bars=bars,
      extended_session=True,
  )
  return df

def fix_symbol(eachange_symbol):
  return eachange_symbol.split(':')[-1] if ':' in eachange_symbol else eachange_symbol

def normalize_dataframe(df):
  df.reset_index(inplace=True)
  df['symbol'] = df['symbol'].apply(fix_symbol)
  if 'datetime' in list(df.columns.values):
    df['dt'] = df['datetime']
    df.drop(columns=['datetime'], inplace=True)
  return df

# with engine.begin() as conn:
#   truncate_table(conn, 'ticks')

for asset in assets:
  symbol    = asset['symbol']
  exchange  = asset['exchange']
  print(f"Processing {exchange}:{symbol}")
  try:
    df = fetch_ticks(symbol, exchange, bars=7*24) # 7*24
    print(f"  count: {df.shape[0]}")
    df = normalize_dataframe(df)
    df.to_sql('daily_ticks', engine, if_exists='append', index=False, method=insert_on_conflict_nothing_ticks)
  except Exception as e:
    print(f"Exception {exchange}:{symbol}")
    print(str(e))


Processing NASDAQ:ADP
  count: 168
Processing NYSE:CVNA
  count: 168
Processing NASDAQ:EUR.USD


ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Exception NASDAQ:EUR.USD
'NoneType' object has no attribute 'shape'
Processing NASDAQ:GCT
  count: 168
Processing NASDAQ:META
  count: 168
Processing NYSE:PAYC
  count: 168
Processing NASDAQ:SBUX
  count: 168
